In [ ]:
!pip install openai langchain langchain-community langchain-openai
!pip install -q langchain-google-genai google-generativeai langchain-core
!pip install wikipedia google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=ddb51c3a79c6944b96df7cc3910c8e2c113740ccddeda55dfb59e07394817ccd
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [ ]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, AIMessage  # msg & res dtype for comm b/w you and LLM

os.environ['GOOGLE_API_KEY'] = userdata.get('Gemini-First-API-Key')

In [ ]:
# Class	            What it Represents	                    Example

# HumanMessage	    User input to the model	                HumanMessage(content="Hello!")
# AIMessage	        Model’s response	                      AIMessage(content="Hi there!")
# SystemMessage	    System-level instruction or role setup	SystemMessage(content="You are a helpful assistant.")
# ChatMessage	      Generic role-based message	            ChatMessage(role="user", content="Hey!")
# BaseMessage	      Abstract base class for all messages	  (used internally)
# BaseOutputParser	Defines how to parse model outputs	    e.g., extracting JSON or numbers
# LLMResult	        Wraps the model’s full response data	  includes generations, metadata, etc.
# Document	        A chunk of text + metadata              Document(page_content="...", metadata={...})

In [ ]:
# llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=2.0)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.0, convert_system_message_to_human=True)

prompt = "Suggest one creative name for a modern American restaurant."


# messages = [
#     SystemMessage(content="You are a naming assistant."),
#     HumanMessage(content="Give me 10 names for a luxury perfume brand."),
# ]

# response = llm.invoke(messages)

response = llm.invoke([HumanMessage(content=prompt)])
# response = model.generate_content(prompt)

print(response.content)

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**The Palate & Plough**

(Combines the refinement of taste with the roots of agriculture and American land.)


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template="Suggest one creative name for a modern {cuisine} restaurant.",
)

prompt_template_name.format(cuisine="Latvian")


'Suggest one creative name for a modern Latvian restaurant.'

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)

chain.run("Pakistani")

/tmp/ipython-input-57772001.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_name)
/tmp/ipython-input-57772001.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Pakistani")
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Here\'s one creative name for a modern Pakistani restaurant:\n\n**Saffron & Steel**\n\n**Why it works:**\n*   **Saffron:** Evokes rich, vibrant Pakistani flavors, warmth, and traditional culinary excellence.\n*   **Steel:** Suggests a sleek, modern, sophisticated, and urban aesthetic, contrasting beautifully with the traditional "saffron."\n*   **Imagery:** It paints a picture of a restaurant that respects its roots while embracing contemporary design and culinary innovation.\n*   **Memorability:** It\'s unique, concise, and easy to remember.'

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.0, convert_system_message_to_human=True)

prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template="Suggest one creative name for a modern {cuisine} restaurant. No description, name only",
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}. Return as items under each category with prices in pkr and also each item description",
)

food_item_chain = LLMChain(llm=llm, prompt=prompt_template_items)

food_item_chain.run(name_chain.run("Pakistani"))

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Welcome to Saffron & Clay, a culinary journey where the ancient warmth of spice meets the rustic charm of earthenware cooking. Our menu is crafted to delight your senses with elevated traditional flavors, infused with modern elegance and the soulful essence of slow-cooked perfection.\n\n---\n\n## Saffron & Clay Menu\n\n**A Culinary Fusion of Tradition and Refinement**\n\n---\n\n### **Starters & Small Plates**\n*(Perfect for beginning your journey or sharing)*\n\n1.  **Saffron & Mint Paneer Tikka Skewers**\n    *   **Price:** PKR 1,250\n    *   **Description:** Cubes of artisanal paneer marinated in a vibrant saffron, fresh mint, and yogurt blend, then perfectly grilled in our clay oven until tender and smoky. Served with a tangy green apple chutney.\n\n2.  **Crispy Okra Fritters (Bhindi Kurkuri)**\n    *   **Price:** PKR 950\n    *   **Description:** Thinly sliced okra lightly coated in a spiced gram flour batter and deep-fried to a delightful crisp. A popular street food elevated, se

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_item_chain], verbose=True)
response = chain.run("Pakistani")
response



> Entering new SimpleSequentialChain chain...


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The Indus Canvas


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Welcome to **The Indus Canvas**, where culinary artistry meets the rich tapestry of Pakistani flavors. Our menu is a curated journey, blending traditional recipes with modern techniques and artistic presentation, inviting you to savor each dish as a masterpiece.

---

### The Indus Canvas Menu

#### I. Prelude to the Palette (Appetizers)

1.  **Saffron & Mint Golgappay Shots**
    *   **Price:** PKR 950
    *   **Description:** Crispy semolina spheres filled with spiced chickpeas and potatoes, served with three vibrant shots: tangy tamarind, refreshing mint, and delicate saffron-infused pani. A playful and invigorating start.

2.  **Lahori Fish Tikki with Dill Raita**
    *   **Price:** PKR 1,250
    *   **Description:** Flaky pan-seared river sole fish cakes, delicately spiced with ginger, green chili, and a hint of ajwain. Served with a cool, creamy dill and cucumber raita, celebrating the flavors of the Indus River.

3.  **Smoked Aubergine & Feta Croquettes**
    *   **Price:** PKR 

"Welcome to **The Indus Canvas**, where culinary artistry meets the rich tapestry of Pakistani flavors. Our menu is a curated journey, blending traditional recipes with modern techniques and artistic presentation, inviting you to savor each dish as a masterpiece.\n\n---\n\n### The Indus Canvas Menu\n\n#### I. Prelude to the Palette (Appetizers)\n\n1.  **Saffron & Mint Golgappay Shots**\n    *   **Price:** PKR 950\n    *   **Description:** Crispy semolina spheres filled with spiced chickpeas and potatoes, served with three vibrant shots: tangy tamarind, refreshing mint, and delicate saffron-infused pani. A playful and invigorating start.\n\n2.  **Lahori Fish Tikki with Dill Raita**\n    *   **Price:** PKR 1,250\n    *   **Description:** Flaky pan-seared river sole fish cakes, delicately spiced with ginger, green chili, and a hint of ajwain. Served with a cool, creamy dill and cucumber raita, celebrating the flavors of the Indus River.\n\n3.  **Smoked Aubergine & Feta Croquettes**\n    *

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.0, convert_system_message_to_human=True)

prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template="Suggest one creative name for a modern {cuisine} restaurant. No description, name only",
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")


prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}. Return as items under each category with prices in {cuisine} price and also each item description",
)

food_item_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")



In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_item_chain],
    input_variables=["cuisine"],
    output_variables=["restaurant_name", "menu_items"],
    verbose=True,
)

chain({
    'cuisine': 'Mongolian'
})



> Entering new SequentialChain chain...


/tmp/ipython-input-2139185515.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



> Finished chain.


{'cuisine': 'Mongolian',
 'restaurant_name': 'Tenger Table',
 'menu_items': 'Welcome to **Tenger Table**, where the vastness of the Mongolian sky meets the richness of the earth on your plate. Our menu is a culinary journey inspired by the nomadic spirit, infused with modern techniques and the finest local and seasonal ingredients. Each dish is crafted to elevate your senses, inviting you to savor the essence of Mongolia with a celestial touch.\n\n---\n\n### Tenger Table Menu\n\n#### A Journey Under the Sky (Appetizers)\n\n*   **Steppe Green Salad**\n    *   ₮28,000\n    *   A vibrant medley of foraged wild greens and herbs, tender microgreens, toasted pine nuts, dried cranberries, and a bright sea buckthorn-honey vinaigrette. A fresh taste of the open grasslands.\n*   **Gobi Beef Tartare**\n    *   ₮35,000\n    *   Finely hand-cut prime Mongolian beef tenderloin, seasoned with smoked paprika, fermented black garlic aioli, capers, and a hint of chili, served with crisp flatbread cracke

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("When was donald trump born? What is his age in 2025?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Action: wikipedia
Action Input: Donald Trump birth date
Observation: Page: Donald Trump and fascism
Summary: There has been significant academic and political debate about whether Donald Trump, the 45th and 47th president of the United States, can be considered a fascist, especially during his 2024 presidential campaign and second term as president. 
A number of prominent scholars, former officials and critics have drawn comparisons between him and fascist leaders with respect to authoritarian actions and rhetoric, while others have rejected the label.
Trump has supported political violence against opponents; many academics cited Trump's involvement in the January 6 United States Capitol attack as an example of fascism. Trump has been accused of racism and xenophobia with respect to his rhetoric about illegal immigrants and his policies of mass deportation and family separation. Trump has a large, dedicated following sometimes referred to as a cult of personality. Trump and his allies'

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Action: wikipedia
Action Input: Donald Trump
Observation: Page: Donald Trump
Summary: Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who is the 47th president of the United States. A member of the Republican Party, he served as the 45th president from 2017 to 2021.
Born into a wealthy family in New York City, Trump graduated from the University of Pennsylvania in 1968 with a bachelor's degree in economics. He became the president of his family's real estate business in 1971, renamed it The Trump Organization, and began acquiring and building skyscrapers, hotels, casinos, and golf courses. He also launched side ventures, many licensing the Trump name, and filed for six business bankruptcies in the 1990s and 2000s. From 2004 to 2015, he hosted the reality television show The Apprentice, bolstering his fame as a billionaire. Presenting himself as a political outsider, Trump won the 2016 presidential election against Democratic Party no

...

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Observation: Answer: 79
Thought:

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I now know the final answer
Final Answer: Donald Trump was born on June 14, 1946. His age in 2025 will be 79.

> Finished chain.


'Donald Trump was born on June 14, 1946. His age in 2025 will be 79.'

In [ ]:
from google.colab import userdata
os.environ['SERPAPI_API_KEY'] = userdata.get('SerpApi')


llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.0, convert_system_message_to_human=True)

tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("What was GDP of US in 2024 and add it to GDP of Pakistan?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Action: Search
Action Input: GDP of US in 2024 (estimated)
Observation: ['Real GDP increased 2.8 percent in 2024 (from the 2023 annual level to the 2024 annual level), the same as previously estimated. The increase in ...', 'Current-dollar nominal GDP increased by 6.04 percent annualized, or $443.616 billion, to a level of $30.486 trillion, the current size of the U.S. economy. The ...', 'Real gross domestic product (GDP) increased at an annual rate of 3.8 percent in the second quarter of 2025 (April, May, and June), according to the third ...', 'According to the World Bank, the U.S. accounted for 14.8% of the global aggregate GDP in 2024 in purchasing power parity terms and 26.2% in nominal terms.', 'The gross domestic product of the United States in 2024 amounted to around 29.18 trillion U.S. dollars.', 'Real GDP growth in 2023 and 2024 was unchanged at 2.9% and 2.8%, respectively. And the slower trajectory for the US economy remained intact with economic ...', 'The GDPNow model esti

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought:I need to find the GDP of the US in 2024 and the GDP of Pakistan in 2024 and then add them together. I will use the Search tool for each GDP.

Action: Search
Action Input: US GDP 2024 forecast
Observation: [{'title': 'Mapped: Every Country’s GDP Growth Forecast for 2025', 'link': 'https://www.visualcapitalist.com/mapped-every-countrys-gdp-growth-forecast-for-2025/', 'source': 'Visual Capitalist', 'source_logo': 'https://serpapi.com/searches/690bac91dbb9ff83c47fd4e4/images/633d8bbdfd36bb4d3ccb7bed83ac20ed98f742f3364107ef35d19b80528b7d01.png', 'date': '5 hours ago', 'thumbnail': 'https://serpapi.com/searches/690bac91dbb9ff83c47fd4e4/images/633d8bbdfd36bb4d3170630a15240bc2ef125f320f88cb39.jpeg'}, {'title': 'Mapped: GDP Growth by U.S. State (1998-2024)', 'link': 'https://www.visualcapitalist.com/gdp-growth-by-u-s-state-map-1990-2024/', 'source': 'Visual Capitalist', 'source_logo': 'https://serpapi.com/searches/690bac91dbb9ff83c47fd4e4/images/633d8bbdfd36bb4d4a82aa090afb09d8d4d927dc

...

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Observation: Answer: 29356
Thought:

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought:I have the GDP of the US in 2024 (approximately $29.01 trillion, which is $29,010 billion) and the GDP of Pakistan in 2024 (approximately $346 billion). I have added them using the calculator.
Final Answer: The estimated combined GDP of the US and Pakistan in 2024 is $29,356 billion (or $29.356 trillion).

> Finished chain.


'The estimated combined GDP of the US and Pakistan in 2024 is $29,356 billion (or $29.356 trillion).'